In [1]:
from datasets import get_dataset_config_names
from datasets import load_dataset
from collections import defaultdict
from datasets import DatasetDict
import pandas as pd
from collections import Counter
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Multilingual NER
Los multilingual transformers han sido entrenados con corpus con múltiples lenguajes (más de 100), lo que permite que puedan realizar *zero-shot-cross-lingual transfer*.

En este capítulo usaremos XLM-RoBERTa (XLM-R), entrenado con un dataset de 2.5 TBs de texto, y aplicaremos fine-tuning para aplicar NER en varios idiomas.

NER puede usarse para:
- Obtener información de los documentos de una compañía.
- Aumentar la calidad de los motores de búsqueda.
- Construir una base de datos estructurada a partir de un corpus.

# Dataset
El dataset a utilizar contiene artículos de Wikipedia en alemán, francés e inglés. Cada artículo está anotado como LOC, PER y ORG.

In [2]:
# Queremos cargar la configuración PAN-X del dataset XTREME, cargamos dicho dataset y comprobamos todas sus configuraciones
xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} configurations")

XTREME has 183 configurations


In [3]:
# Cargamos aquellas configuraciones que empiezan por "PAN"
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

In [4]:
# Cargamos la configuración que nos interesa
load_dataset("xtreme", name="PAN-X.de")

Using the latest cached version of the module from C:\Users\jcav\.cache\huggingface\modules\datasets_modules\datasets\xtreme\29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4 (last modified on Wed Mar 15 09:15:50 2023) since it couldn't be found locally at xtreme., or remotely on the Hugging Face Hub.


KeyboardInterrupt: 

In [ ]:
# Vamos a cargar los datasets en alemán, francés, italiano e inglés en una proporción similar a su porcentaje de uso en Suiza
langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]

# Return a DatasetDict if a key does not exist
panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    # Load monolingual corpus
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    # Shuffle and downsample each split according to the proportions
    for split in ds:
        panx_ch[lang][split] = (
            ds[split]
            .shuffle(seed=0)
            .select(range(int(frac * ds[split].num_rows)))
        )

In [ ]:
pd.DataFrame({lang: [panx_ch[lang]["train"].num_rows] for lang in langs}, 
             index=["Number of training examples"])

Al tener más ejemplos en alemán que en los otros idiomas combinados, lo usaremos como punto de partida para aplicar *zero-shot-cross-lingual* transfer a francés, italiano e inglés. Mostramos algunos ejemplos en el corpus alemán:

In [ ]:
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}: {value}")

In [ ]:
# Modificamos las ner_tags para que sean más legibles
for key, value in panx_ch['de']['train'].features.items():
    print(f"{key}: {value}")

In [ ]:
# Extraemos las ner tags
tags = panx_cg["de"]["train"].features["ner_tags"].feature
print(tags)

Utilizamos el método `ClassLabel.int2str()` para crear una nueva columna con los nombres de cada clase. 

In [ ]:
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

In [ ]:
panx_de = panx_ch["de"].map(create_tag_names)

Vemos como se alinean las tags del primer ejemplo del training set de manera más gráfica:

In [ ]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]],
             ["Tokens", "Tags"])

Comprobamos que las tags no están desbalanceadas:

In [ ]:
split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1

pd.DataFrame.from_dict(split2freqs, orient="index")

# Multilingual Transformers
La principal diferencia con los transformers entrenados en un solo idioma reside en que se ha empleado un corpus formado por documentos escritos en varios idiomas. A pesar de no recibir explícitamente información para diferenciar entre los idiomas, las representaciones linguísticas resultantes pueden generalizar entre los distintos idiomas.

Los multilingual transformer models suelen evaluarse de tres formas:
- *en*: Fine-tune con datos de entrenamiento en inglés y evaluar en el test set de cada idioma.
- *each*: Fine-tune y evaluar en datos de test monolinguísticos para medir el rendimiento por idioma.
- *all*: Fine-tune en todos los datos de entrenamiento y evaluar en cada lenguaje del conjunto de test.

# A closer look at Tokenization
En lugar de usar un tokenizer de tipo **WordPiece**, XLM-R utiliza un tokenizer de tipo **SentencePiece**, que es entrenado en texto plano en cien idiomas.

Comparemos el WordPiece de BERT con el SentencePiece de XLM-R

In [ ]:
bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

In [ ]:
text = "Jack Sparrow loves New York!"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

## Tokenizer pipeline
Cuatro pasos:
1. **Normalization:** Incluye operaciones como eliminar espacios en blanco, tildes, lowercasing y convertir a unicode.

1. **Pretokenization:** Divide el texto en unidades más pequeñas (generalmente palabras) que establecen un upper bound para los tokens que obtendremos en pasos subsecuentes.

1. **Tokenizer model:** El tokenizer aplica subdivisiones de las palabras obtenidas en el paso anterior. Esta es la parte del tokenizer que se entrena en el corpus. El rol del modelo consiste en dividir las palabras en subpalabras para dividir el tamaño del vocabulario y reducir el número de tokens out-of-scope. En este punto, tenemos una lista de IDs en lugar de strings.

1. **Postprocessing:** Este es el último paso de la tokenization pipeline, en la que se añaden algunas transformaciones adicionales a la lista de tokens, como añadir los tokens especiales [CLS] y [SEP]. Esta secuencia puede usarse como entrada para el modelo.

## SentencePiece Tokenizer
SentencePiece tokenizer está basado en un tipo de subword segmentation llamado Unigram que codifica (encode) cada texto del input como una secuencia de caracteres unicode. Esto resulta especialmente útil si el corpus está escrito en múltiples idiomas por dos motivos:

- Permite que el modelo sea agnóstico a tildes, puntuaciones o falta de espacios en blanco en ciertos idiomas (e.g., japonés).
- Los espacios en blancos son asignados al símbolo Unicode U+2581, o el caracter _, lo que permite detokenizar una secuencia sin ambigüedades y sin tener que depender de tokenizers específicos del dominio.

In [ ]:
"".join(xlmr_tokens).replace(u"\u2581", " ")

Vamos a crear nuestro propio ejemplo de como codificar (encode) un simple ejemplo de forma que sea procesable por NER. Podríamos cargar una head preentrenada. Pero **en esta demo crearemos nuestra propia cabeza desde cero (en un ejemplo real, utilizar una cabeza ya entrenada).**

# Transformers para NER